In [ ]:
tbma<-function(formula,train,test,prediction_type="point",percentile=c(0.25,0.5,0.75),group_id=NULL,horizon=nrow(train),splitrule="extratrees",always_split_variables=NULL,min_node_size=5,max_depth=NULL,num_trees=100,ma_order=2,mtry=round(sqrt(ncol(train)))){

  Order<-NULL
  response<-NULL
  basenames<-NULL
  value<-NULL
  id<-NULL
  variable<-NULL
  candidate_forecast<-NULL

  if(length(group_id)>1){print("pelase enter only one column name as group identity")}



  #subset the data according to the formula
  train<-as.data.table(model.frame(formula,data = train))
  test<-as.data.table(model.frame(formula,data = test))
  train[,Order:=1:nrow(train)]
  test[,Order:=(nrow(train)+1):(nrow(test)+nrow(train))]

  #give the gruop_id column the "group_id" name
  colnames(train)[colnames(train)==group_id] <- "group_id"
  colnames(test)[colnames(test)==group_id] <- "group_id"


  #fit decision trees
  if(is.null(group_id)){
    fit=ranger(formula, data = train, num.trees = num_trees, importance = "impurity", always.split.variables=c(always_split_variables),replace=TRUE,
               splitrule=splitrule,mtry = mtry, keep.inbag=TRUE,num.random.splits=5,num.threads=4,min.node.size = min_node_size,max.depth = max_depth)
  } else{
    fit=ranger(formula, data = train, num.trees = num_trees, importance = "impurity", always.split.variables=c("group_id",always_split_variables),replace=TRUE,
               splitrule=splitrule,mtry = mtry, keep.inbag=TRUE,num.random.splits=5,num.threads=4,min.node.size = min_node_size,max.depth = max_depth)
  }

  #prepare data
  temporary<-copy(test)
  response_col_name<-formula[[2]]
  colnames(train)[colnames(train)==response_col_name] <- "response"
  colnames(test)[colnames(test)==response_col_name] <- "response"
  test[,response:=NA]

  #determine the terminal nodes of inbag train observations
  inbag_terminals=predict(fit,train,predict.all=TRUE,type='terminalNodes')$predictions
  inbag = data.table(do.call(cbind, fit$inbag.counts))
  inbag[inbag==0]=NA
  inbag[!is.na(inbag)]=1
  inbag_terminals=inbag_terminals*as.matrix(inbag)
  baseline_train=data.table(train[,list(Order,response,group_id)],inbag_terminals)

  #apply horizon parameter (select the last n observation from the train set)
  if(!is.null(horizon)){
    baseline_train<-tail(baseline_train,n=horizon)
  }


  #determine the terminal nodes of test observations
  test_terminals=predict(fit,test,predict.all=TRUE,type='terminalNodes')$predictions
  baseline_test=data.table(test[,list(Order,response,group_id)],test_terminals)

  #combine the infromation of leaf nodes of inbah ang test observations
  baseline=rbind(baseline_train,baseline_test)
  if(is.null(group_id)){
    baseline=melt(baseline,id.vars=c('Order',"response"),measure_vars=basenames[!(basenames %in% c('Order',"response","group_id"))])
  }else{
    baseline=melt(baseline,id.vars=c('Order',"response","group_id"),measure_vars=basenames[!(basenames %in% c('Order',"response","group_id"))])
  }

  baseline=baseline[order(Order)]
  baseline=baseline[!is.na(value)]

  #determine groups based on tree nodes and group identity parameter
  if(is.null(group_id)){
    setDT(baseline)[, id := .GRP, by=list(variable,value)]
  } else{
    setDT(baseline)[, id := .GRP, by=list(variable,value,group_id)]}

  #apply moving average
  baseline[,candidate_forecast:=RcppRoll::roll_meanr(c(response),ma_order,fill = NA,na.rm = F),by=id]
  baseline[,candidate_forecast := shift(candidate_forecast, fill = NA,n=1), by = id]

  #seperate the test data from the combined inbag and test data
  test_results<-baseline[Order %in% test$Order ]

  #fill some of the candidate forecasts with the same id candidate forecasts
  test_results=test_results[order(Order)]
  test_results[,candidate_forecast:=na.locf(candidate_forecast, na.rm = FALSE),by=id]

  if(prediction_type=="point"){
    #find the median of the candidate forecasts for each test observation
    result=test_results[,list(prediction=median(candidate_forecast,na.rm=T)),by=list(Order,response)]

    #order the test data
    result[,response:=NULL]
    result=merge(temporary,result,by="Order")
    result=result[order(Order)]
  }



  if(prediction_type=="probabilistic"){
    result=test_results[,list(prediction=quantile(candidate_forecast,percentile[1],na.rm = TRUE)),by=list(Order)]
    colnames(result)[colnames(result)=="prediction"] <- as.character(paste(100*percentile[1], 'percentile', sep = '_'))
    if(length(percentile>1)){
      for(i in 2:length(percentile)){
        #find the percentile of the candidate forecasts for each test observation
        temp1<-test_results[,list(prediction=quantile(candidate_forecast,percentile[i],na.rm = TRUE)),by=list(Order)]
        result[,as.character(paste(100*percentile[i], 'percentile', sep = '_'))]<-temp1$prediction
      }
    }
    #order the test data
    result=merge(temporary,result,by="Order")
    result=result[order(Order)]
  }

  return(result)

}